<a href="https://colab.research.google.com/github/ohyunju/projects/blob/ohyunju-patch-1/project1/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 프로젝트에 꼭 포함되야 하는 내용
1. 지역에 따라서 선호하는 게임 장르가 다를까
2. 연도별 게임의 트렌드가 있을까
3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스


---



## 학습 계획
* 1일차 : EDA(데이터 전처리) 및 데이터를 가지고 할 수 있는 계획 수립
* 2일차 : Hypothesis Test, CI, Bayesian, DR, PCA, Cluster 및 시각화
* 3일차 : Hypothesis Test, CI, Bayesian, DR, PCA, Cluster 및 시각화
* 4일차 : 스크립트 제작 및 영상물 촬영


---



In [41]:
import pandas as pd

url='https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
df = pd.read_csv(url)

# Unnamed: 0 는 단순 인덱스넘버이므로 삭제
df = df.drop('Unnamed: 0', axis = 1) 
df.isnull().sum()
# 'Year' 271 / 'Genre' 50 / 'Publisher' 58 의 널값 존재, 결측치 드랍
df_man=df.dropna()
# 칼럼명 'Name','Platform','Year','Genre','Publisher','NA_Sales','EU_Sales','JP_Sales','Other_Sales' 

print(df_man.info())
df_man.head() #(16241 rows × 9 columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB
None


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


## 계획 수립
1. 지역에 따라서 선호하는 게임 장르가 다를까 
> **파이그래프**로 각 지역에 따른 분포를 보여주기, 4개의 지역에 따른 선호 장르를 차이를 따지기 위해 **ANOVA**사용 
2. 연도별 게임의 트렌드가 있을까 (게임의 트렌드라는 말을 장르로 인식) 
> 4가지의 Sales 칼럼을 모두 모은 Total_Sales 칼럼을 만들어, *x=출시연도 & y=Total_Sales & hue=장르*로 **선그래프**로시각화, 연도와 토탈세일에 대한 차이가 있는지 확인하기 위해 **T-test** 나 **chi-square-test**, **chi2_contingency**사용
3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스 
> 위에서 만든 Total_Sales 라는 칼럼을 이용, **누적그래프**를 이용해서 출고량이 제일 많다는 것 보여주기, 출고량 상위 30가지 항목이용
4. 연도별 플랫폼에도 트렌드가 있을까 
> 4가지의 Sales 칼럼을 모두 모은 Total_Sales 칼럼을 만들어, *x=출시연도 & y=Total_Sales & hue=플랫폼*으로 **선그래프**로시각화, 연도와 토탈세일에 대한 차이가 있는지 확인하기 위해 **T-test** 나 **chi-square-test**, **chi2_contingency**사용
5. 제일 선호되는 회사와 지역에 따라서 선호하는 회사가 다를까 
> 전체적으로 랭킹 상위30 (575개의 회사), **파이그래프**로 각 지역에 따른 분포를 보여주기, 4개의 지역에 따른 회사 선호 장르를 차이를 따지기 위해 **ANOVA**사용(자국의 회사를 선호하는지?, 자국인지는 구글링 해보기)
6. 출시이후 평균 판매량은 어떤지 확인하기, 전체적으로도, 지역별로 그룹해서도, 또한 순위 재평가(상위 30개 게임)
> 출시이후 평균 판매량=총판매량/(지금년도-출시년도+1) 라는 수식을 사용(지금의 판매량은 누적이기 때문에 늦게 출시된 제품의 판매량이 떨어질 수 밖에 없다고 판단), 꾸준히 인기가 있는 게임이 있다면 원인은 무엇일까?

# + DR, PCA, Cluster 를 활용할 수 있는 방안 생각하기

### 결과 
이러한 분석들을 통해 전세계를 대상으로한 게임을 출시하려고 한다면 
  * 지역에 따라 게임의 장르, 플랫폼을 다르게 설정해야 하는지
  * 지금 트렌드의 장르로 개발하는 것이 맞는지 (연도별 장르의 추세를 보여줄 수 있는 선 그래프 활용, 약 40년의 기간을 5년의 기간으로 쪼개고 그동안의 장르(12개중 상위 3개씩) 변동 보여주는데 바 그래프 이용)
  * 위 질문에 대한 타탕성의 결과 어떤 게임을 출시해야 한다.


In [ ]:
header=['Platform','Year','Genre','Publisher']
for i in header:
  df_check=df_man[i]
  df_check=df_check.drop_duplicates()
  print('길이',len(df_check))
  print(df_check)

# 플랫폼에 숫자(2600)가 있길래 잘못 입력인줄 알았는데 구글에 확인하니 이런 플랫폼이 존재 (아타리2600)
# 연도에 이상한 값 있음

In [43]:
#'Year' 칼럼 100초과로 잡아줌
df1=df_man[df_man.Year >100]
df1.describe()
df1 #(16141 rows × 9 columns)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [ ]:
# Sales 값 들에 대한 데이터 전처리 (K,M)
